In [ ]:
import time
import numpy as np 
from zmqRemoteApi import RemoteAPIClient

In [ ]:
def degree_To_radian(degree):
    return (degree * np.pi/180)

def FK(th,alp,a,d):
    
    Cth=np.cos(th); Sth=np.sin(th); Calp=np.cos(alp); Salp=np.sin(alp)
    T=np.array([
                    [Cth, -Sth*Calp, Sth*Salp, a*Cth],
                    [Sth, Cth*Calp, -Cth*Salp , a*Sth],
                    [0,Salp,Calp,d],
                    [0,0,0,1]
               ])
    return np.round(T,2)

In [ ]:
def Calculate_Jacopian_P(Trans_mat_arr):
    
    O_0 = np.zeros([3,1])
    Jp_size = len(Trans_mat_arr)
    Z_0 = np.array([[0,0,1]]).reshape(3,1)
    
    Z_arr = [z[0:3,2] for z in Trans_mat_arr]
    Z_arr.insert(0,Z_0)
    
    O_arr = [o[0:3,3] for o in Trans_mat_arr]
    O_arr.insert(0,O_0)
    
    O_last = O_arr[-1].reshape(3,1)
    
    jp_mat = np.array([
        [np.cross(Z_arr[i].reshape(3,1),(O_last-O_arr[i].reshape(3,1)),axis=0) for i in range(Jp_size)]
    ])
        
    return jp_mat.reshape(3,Jp_size).transpose()
    
def Calculate_Jacopian_r(Trans_mat_arr):
    
    Z_0 = np.array([[0,0,1]])
    Jp_size = len(Trans_mat_arr)
        
    Z_arr = [Trans_mat_arr[z][0:3,2] for z in range(Jp_size-1)]
    Z_arr.insert(0,Z_0)
    
    Jr_mat = np.array([[z.reshape(3,1) for z in Z_arr]])
    
    return Jr_mat.reshape(3,Jp_size).transpose()

def Calculate_J(Trans_mat_arr):
    J_size = len(Trans_mat_arr)
    J = np.array([
                    [Calculate_Jacopian_P(Trans_mat_arr),
                     Calculate_Jacopian_r(Trans_mat_arr)]
                ])
    
    return J.reshape(6,J_size)
    

In [ ]:
def Calculate_Delta_X(Td, T0):
    
    Pd = Td[0:3,3]
    P0 = T0[0:3,3]
    
    Rd = Td[0:3,0:3]
    R0 = T0[0:3,0:3]
    
    I = np.identity(3)
    
    Pos_list = np.array([[d-c for d,c in zip(Pd,P0)]])
    Pos_list = Pos_list.reshape(3,1)
    
    DR = np.matmul(Rd,R0.transpose())
    skew = np.subtract(DR , I)
    
    vec = np.array([[abs(skew[1,2]),abs(skew[0,2]) ,abs(skew[0,1])]])
    vec = vec.reshape(3,1)
    
    Delta_X = np.array([Pos_list,vec])
    
    print("Delta_X = \n",Delta_X)
    
    return Delta_X.reshape(6,1)  

In [ ]:
def Calculate_IK(dX,J,q0):
    # qd = q0 + J * d_X
    qd = numpy.add(q0,np.matmul(J,dX))
    return qd

In [ ]:
error = 0.001
q0 = [90,45,90,45]
alpha = []

Trans_list = [FK(th, ) for th in q0]
J = Calculate_J(Trans_list)
delta_X = Calculate_Delta_X(Trans_list[0],Trans_list[-1])
qd = Calculate_IK(delta_X,J,q0)

In [ ]:
while ~(np.linalg.norm(delta_X) < error):
    
    q0 = qd
    Trans_list = [FK(th, ) for th in q0]
    J = Calculate_J(Trans_list)
    delta_X = Calculate_Delta_X(Trans_list[0],Trans_list[-1])
    qd = Calculate_IK(delta_X,J,q0)

In [ ]:
client = RemoteAPIClient()
sim = client.getObject('sim')

# When simulation is not running, ZMQ message handling could be a bit
# slow, since the idle loop runs at 8 Hz by default. So let's make
# sure that the idle loop runs at full speed for this program:
defaultIdleFps = sim.getInt32Param(sim.intparam_idle_fps)
sim.setInt32Param(sim.intparam_idle_fps, 0)

In [ ]:
# set Object name 
joint_names = [f"joint_{i+1}"  for i in range(6)]
# set Object handel 
joint_handels = [sim.getObjectHandle(joint) for joint in joint_names]
joint_Position = [sim.getJointPosition(handle) for handle in joint_handels]

for handle in joint_handels:
    sim.setObjectInt32Parameter(handle ,2001,~0) #to enable the position control

In [ ]:
client.setStepping(False)
sim.startSimulation()

for (handle,degree) in zip(joint_handels,qd):
    sim.setJointTargetPosition(handle,degree) # degree by radian
    sim.startSimulation()

In [ ]:
sim.stopSimulation()